In [9]:
from pyspark.sql import SparkSession
# Spark 세션 생성
spark = (
    SparkSession.builder
    .appName("olist")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "6g")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)


#seller = spark.read.csv('/app/spark/Olist/raw_data/olist_sellers_dataset.csv', header=True, inferSchema=True)
geo = spark.read.csv('/app/spark/Olist/dim_table/dim_geo.csv',header=True,inferSchema=True)


ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
dim_seller = seller.join(
    geo,
    seller["seller_zip_code_prefix"] == geo["geolocation_zip_code_prefix"],
    how='inner'
)
dim_seller.show(5)

- 위,경도가 geolocation 데이터에서 왔으므로 geoloaction_city로 seller_city를 대체

In [6]:
dim_seller = dim_seller.select(
    'seller_id', 'seller_zip_code_prefix', 'geolocation_city', 'geolocation_state', 'geolocation_region','geolocation_lat','geolocation_lng'
)

dim_seller = dim_seller \
    .withColumnRenamed('geolocation_city', 'seller_city') \
    .withColumnRenamed('geolocation_state', 'seller_state') \
    .withColumnRenamed('geolocation_region', 'seller_region') \
    .withColumnRenamed('geolocation_lat', 'seller_lat') \
    .withColumnRenamed('geolocation_lng', 'seller_lng') 
dim_seller.show(5)

+--------------------+----------------------+-----------+------------+-------------+-------------------+------------------+
|           seller_id|seller_zip_code_prefix|seller_city|seller_state|seller_region|         seller_lat|        seller_lng|
+--------------------+----------------------+-----------+------------+-------------+-------------------+------------------+
|e5cbe890e67949012...|                  1041|  sao paulo|          SP|       남동부|  -23.5443921648681|-46.63949930627844|
|1d503743d2526f03f...|                  1035|  sao paulo|          SP|       남동부|-23.541577961711493|-46.64160722329613|
|2d34636518ba88f53...|                  1039|  sao paulo|          SP|       남동부| -23.54188300998332|-46.63991946670314|
|e5cbe890e67949012...|                  1041|  sao paulo|          SP|       남동부|-23.543539449073783|-46.64057731057763|
|b6c6854d4d92a5f6f...|                  1042|  sao paulo|          SP|       남동부| -23.54487359810152|-46.64030309449483|
+--------------------+-

- dim_sellers 저장

In [7]:
dim_seller.toPandas().to_csv('/app/spark/Olist/dim_table/dim_sellers.csv',index=False)

In [3]:
customer = spark.read.csv('/app/spark/Olist/raw_data/olist_customers_dataset.csv', header=True, inferSchema=True)
dim_customer = customer.join(
    geo,
    customer["customer_zip_code_prefix"] == geo["geolocation_zip_code_prefix"],
    how='inner'
)
dim_customer.show(5)

+--------------------+--------------------+------------------------+-------------+--------------+---------------------------+------------------+------------------+----------------+-----------------+------------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|geolocation_zip_code_prefix|   geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|geolocation_region|
+--------------------+--------------------+------------------------+-------------+--------------+---------------------------+------------------+------------------+----------------+-----------------+------------------+
|6519f59a7687a4661...|324ce23e08a176800...|                    1037|    sao paulo|            SP|                       1037|-23.54562128115268|-46.63929204800168|       sao paulo|               SP|            남동부|
|19ee522d0dd140702...|8e7b6c489f1fe794c...|                    1037|    sao paulo|            SP|                       1037|-23.54

In [4]:
dim_customer = dim_customer.select(
    'customer_id', 'customer_zip_code_prefix', 'geolocation_city', 'geolocation_state', 'geolocation_region','geolocation_lat','geolocation_lng'
)

dim_customer = dim_customer \
    .withColumnRenamed('geolocation_city', 'customer_city') \
    .withColumnRenamed('geolocation_state', 'customer_state') \
    .withColumnRenamed('geolocation_region', 'customer_region') \
    .withColumnRenamed('geolocation_lat', 'customer_lat') \
    .withColumnRenamed('geolocation_lng', 'customer_lng') 
dim_customer.show(5)

+--------------------+------------------------+-------------+--------------+---------------+------------------+------------------+
|         customer_id|customer_zip_code_prefix|customer_city|customer_state|customer_region|      customer_lat|      customer_lng|
+--------------------+------------------------+-------------+--------------+---------------+------------------+------------------+
|6519f59a7687a4661...|                    1037|    sao paulo|            SP|         남동부|-23.54562128115268|-46.63929204800168|
|19ee522d0dd140702...|                    1037|    sao paulo|            SP|         남동부|-23.54562128115268|-46.63929204800168|
|05b005e82f3f2dcc4...|                    1037|    sao paulo|            SP|         남동부|-23.54562128115268|-46.63929204800168|
|4b6cc748c9b98d12c...|                    1046|    sao paulo|            SP|         남동부|-23.54608112703553|-46.64482029837157|
|0c65144301b9eb127...|                    1046|    sao paulo|            SP|         남동부|-23.54

In [5]:
dim_customer.toPandas().to_csv('/app/spark/Olist/dim_table/dim_customers.csv',index=False)

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java s

ConnectionRefusedError: [Errno 111] Connection refused